In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import stft

In [ ]:
#验证测试代码
labeled_data = pd.read_csv("DATA/Labeled_data/0820-0NC-Y/labeled_data.csv")
labeled_data 
# labeled_data.columns.tolist()
# label_0_data = labeled_data[labeled_data['label'] == 1]
# print(label_0_data.head())
# label_counts = labeled_data['label'].value_counts()
# print(label_counts)

In [ ]:
#验证 数据块分类是否正确代码
labeled_data = pd.read_csv("DATA/tactile_dataset.csv")
# labeled_data
# labeled_data.columns.tolist()
# # label_0_data = labeled_data[labeled_data['label'] == 1]
# # print(label_0_data.head())

# 只筛选出 touch_type 为 'ST' 的数据
st_data = labeled_data[labeled_data['touch_type'] == 'ST' ]
# st_data
st_data[st_data['block_id']==100]
# 计算每个 (block_id, touch_type) 组合的出现次数
# st_label_counts = st_data.groupby(['block_id', 'touch_type']).size()
# st_label_counts
# 计算每个 (block_id, touch_type) 组合的出现次数
# label_counts = labeled_data.groupby(['block_id','touch_type']).size()
# label_counts


In [ ]:
# 测试验证代码
import pandas as pd

# 读取CSV文件
file_path = '/Users/shangyu/Documents/GitHub/contactInterpretation-main/tactileGestureDetection-main/DATA/Labeled_data/0724-7DT-S1/labeled_data.csv'
df = pd.read_csv(file_path)

# 指定要显示的行范围
start_row = 408 # 起始行（包含）
end_row = 438  # 结束行（包含）

# 显示从start_row到end_row的行
# print(df.iloc[start_row:end_row + 1])

In [ ]:
np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress = True)
time_data = data['time'].values
fs = 1 / np.mean(np.diff(time_data))  # 计算采样率
#print(time_data)
#print(fs)
print(np.diff(time_data))
print(np.mean(np.diff(time_data)))

In [ ]:
#
# ResNet model+ K-fold Cross-validation:Average accuracy using K-Fold Cross-Validation:  0.2449
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import KFold

# Read data and images
data_path = "../tactileGestureDetection-main/DATA/STFT_images/"
file = os.path.join(data_path, 'image_records.csv')
image_df = pd.read_csv(file)

images = []
labels = []

# Load and preprocess images
for index, row in image_df.iterrows():
    img_path = os.path.join(data_path, row['image_filename'])
    img = Image.open(img_path).convert('RGB')
    img = img.resize((128, 128))#print 一下
    img = np.array(img)
    images.append(img)
    labels.append(row['touch_type'])

images = np.array(images)
labels = np.array(labels)

# Encode labels
label_classes = np.unique(labels)
label_map = {label: idx for idx, label in enumerate(label_classes)}
labels = np.array([label_map[label] for label in labels])
labels = to_categorical(labels, num_classes=len(label_classes))

# Normalize images
images = images / 255.0

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    # horizontal_flip=True,#
    fill_mode='nearest'
)

# K-Fold Cross-Validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []
histories = []  # Store the history of each fold

for train_index, test_index in kf.split(images):
    X_train, X_test = images[train_index], images[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # Initialize the ResNet model
    resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    resnet_base.trainable = True  # Fine-tune the base model

    # Add custom layers on top of ResNet base
    model = Sequential([
        resnet_base,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(0.5),  # Add dropout for regularization
        Dense(len(label_classes), activation='softmax')
    ])

    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Early stopping and learning rate reduction on plateau to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

    # Train model with data augmentation
    history = model.fit(datagen.flow(X_train, y_train, batch_size=16),
                        epochs=20, 
                        validation_data=(X_test, y_test), 
                        callbacks=[early_stopping, reduce_lr], 
                        verbose=0)
    
    # Evaluate model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    accuracies.append(accuracy)
    histories.append(history)

# Print accuracies
print(f"Accuracies are {accuracies}")

# Calculate mean accuracy
mean_accuracy = np.mean(accuracies)
print(f'Average accuracy using K-Fold Cross-Validation: {mean_accuracy:.4f}')

# Plot accuracy
plt.figure(figsize=(12, 4))
plt.plot(histories[0].history['accuracy'], label='Training accuracy')
plt.plot(histories[0].history['val_accuracy'], label='Validation accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()



In [ ]:
#CNN+K-Fold Cross-Validation:Average accuracy using K-Fold Cross-Validation: 0.8571
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import KFold

# Read data and images
data_path = "../tactileGestureDetection-main/DATA/STFT_images/"
file = os.path.join(data_path, 'image_records.csv')
image_df = pd.read_csv(file)

images = []
labels = []

# Load and preprocess images
for index, row in image_df.iterrows():
    img_path = os.path.join(data_path, row['image_filename'])
    img = Image.open(img_path).convert('RGB')
    img = img.resize((128, 128))
    img = np.array(img)
    images.append(img)
    labels.append(row['touch_type'])

images = np.array(images)
labels = np.array(labels)

# Encode labels
label_classes = np.unique(labels)
label_map = {label: idx for idx, label in enumerate(label_classes)}
labels = np.array([label_map[label] for label in labels])
labels = to_categorical(labels, num_classes=len(label_classes))

# Normalize images
images = images / 255.0

# Initialize K-Fold Cross-Validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []
histories = []  # Store the history of each fold

# Iterate through each fold
for train_index, test_index in kf.split(images):
    X_train, X_test = images[train_index], images[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    # Simple CNN model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(len(label_classes), activation='softmax')
    ])

    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train model with early stopping
    history = model.fit(X_train, y_train, epochs=20, batch_size=16, verbose=0, validation_data=(X_test, y_test), callbacks=[early_stopping])
    
    # Evaluate model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    accuracies.append(accuracy)
    histories.append(history)

# Print accuracies
print(f"Accuracies are {accuracies}")

# Calculate mean accuracy
mean_accuracy = np.mean(accuracies)
print(f'Average accuracy using K-Fold Cross-Validation: {mean_accuracy:.4f}')

# Plot accuracy
plt.figure(figsize=(12, 4))
plt.plot(histories[0].history['accuracy'], label='Training accuracy')
plt.plot(histories[0].history['val_accuracy'], label='Validation accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
# VGG16 model + K-Fold Cross-Validation：Average accuracy using K-Fold Cross-Validation 0.7787
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import KFold

# Read data and images
data_path = "/Users/shangyu/Documents/GitHub/contactInterpretation-main/tactileGestureDetection-main/DATA/STFT_images/"
file = data_path + 'image_records.csv'
image_df = pd.read_csv(file)

images = []
labels = []

# Load and preprocess images
for index, row in image_df.iterrows():
    img_path = os.path.join(data_path, row['image_filename'])
    img = Image.open(img_path).convert('RGB')
    img = img.resize((128, 128))
    img = np.array(img)
    images.append(img)
    labels.append(row['touch_type'])

images = np.array(images)
labels = np.array(labels)

# Encode labels
label_classes = np.unique(labels)
label_map = {label: idx for idx, label in enumerate(label_classes)}
labels = np.array([label_map[label] for label in labels])
labels = to_categorical(labels, num_classes=len(label_classes))

# Normalize images
images = preprocess_input(images)

# K-Fold Cross-Validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []

for train_index, test_index in kf.split(images):
    X_train, X_val = images[train_index], images[test_index]
    y_train, y_val = labels[train_index], labels[test_index]

    # Data augmentation
    train_data = ImageDataGenerator(
        rotation_range=20, 
        width_shift_range=0.1, 
        height_shift_range=0.1, 
        shear_range=0.1, 
        zoom_range=0.1, 
        horizontal_flip=True
    )
    val_data = ImageDataGenerator()

    train_generator = train_data.flow(X_train, y_train, batch_size=32)
    val_generator = val_data.flow(X_val, y_val, batch_size=32)

    # Using pre-trained VGG16 model
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    base_model.trainable = False  # Freeze the convolutional base

    model = Sequential([
        base_model,
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(len(label_classes), activation='softmax')
    ])

    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    # Define callbacks
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train model
    fitted = model.fit(train_generator, epochs=30, validation_data=val_generator, callbacks=[reduce_lr, early_stopping], verbose=0)

    # Evaluate model
    loss, accuracy = model.evaluate(val_generator, verbose=0)
    accuracies.append(accuracy)

# Print accuracies
print(f"Accuracies are {accuracies}")

# Calculate mean accuracy
mean_accuracy = np.mean(accuracies)
print(f'Average accuracy using K-Fold Cross-Validation: {mean_accuracy:.4f}')

# Plot accuracy
plt.figure(figsize=(12, 4))
plt.plot(fitted.history['accuracy'], label='Training accuracy')
plt.plot(fitted.history['val_accuracy'], label='Validation accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
# Simple CNN model
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

# Read data and images
data_path = "/Users/shangyu/Documents/GitHub/contactInterpretation-main/tactileGestureDetection-main/DATA/STFT_images/"
file = os.path.join(data_path, 'image_records.csv')
image_df = pd.read_csv(file)

images = []
labels = []

# Load and preprocess images
for index, row in image_df.iterrows():
    img_path = os.path.join(data_path, row['image_filename'])
    img = Image.open(img_path).convert('RGB')
    img = img.resize((128, 128))
    img = np.array(img)
    images.append(img)
    labels.append(row['touch_type'])

images = np.array(images)
labels = np.array(labels)

# Encode labels
label_classes = np.unique(labels)
label_map = {label: idx for idx, label in enumerate(label_classes)}
labels = np.array([label_map[label] for label in labels])
labels = to_categorical(labels, num_classes=len(label_classes))

# Split data
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.3, random_state=42)

# Normalize images
X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)

# Simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_classes), activation='softmax')
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), callbacks=[reduce_lr, early_stopping])

# Visualize training process
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Evaluate model
loss, accuracy = model.evaluate(X_val, verbose=0)
print(f'Validation accuracy: {accuracy:.4f}')
